In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk , string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
movies = pd.read_csv('/content/TMDB_movie_dataset_v11.csv')

In [12]:
movies.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [13]:
movies.isnull().sum()

,0
id,0
title,12
vote_average,0
vote_count,0
status,0
release_date,147775
revenue,0
runtime,0
adult,0
backdrop_path,775101


In [14]:
movies.shape

(1067188, 24)

In [15]:
movies['backdrop_path'][0]

'/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg'

In [16]:
movies['poster_path'][0]

'/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg'

In [17]:
movies['homepage'][0]

'https://www.warnerbros.com/movies/inception'

In [19]:
movies['production_companies'][0]

'Legendary Pictures, Syncopy, Warner Bros. Pictures'

In [20]:
movies['production_countries'][0]

'United Kingdom, United States of America'

In [18]:
movies.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [21]:
movies.drop(['backdrop_path','homepage','production_companies','production_countries','original_title','imdb_id'],axis=1,inplace=True)

In [24]:
movies.drop(['spoken_languages'],axis=1,inplace=True)

In [25]:
movies.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,original_language,overview,popularity,poster_path,tagline,genres,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,160000000,en,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,165000000,en,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,185000000,en,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,237000000,en,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,220000000,en,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure","new york city, superhero, shield, based on com..."


In [32]:
movies.dropna(inplace=True)

In [33]:
movies.shape

(66583, 17)

In [35]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66583 entries, 0 to 1067169
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 66583 non-null  int64  
 1   title              66583 non-null  object 
 2   vote_average       66583 non-null  float64
 3   vote_count         66583 non-null  int64  
 4   status             66583 non-null  object 
 5   release_date       66583 non-null  object 
 6   revenue            66583 non-null  int64  
 7   runtime            66583 non-null  int64  
 8   adult              66583 non-null  bool   
 9   budget             66583 non-null  int64  
 10  original_language  66583 non-null  object 
 11  overview           66583 non-null  object 
 12  popularity         66583 non-null  float64
 13  poster_path        66583 non-null  object 
 14  tagline            66583 non-null  object 
 15  genres             66583 non-null  object 
 16  keywords           66583 

In [36]:
clear_dataset = movies

In [37]:
clear_dataset.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'original_language',
       'overview', 'popularity', 'poster_path', 'tagline', 'genres',
       'keywords'],
      dtype='object')

In [ ]:
clear_dataset['http']='https://image.tmdb.org/t/p/w500'

In [ ]:
clear_dataset['http']+=clear_dataset['poster_path']

In [ ]:
clear_dataset.drop(['poster_path'],axis=1,inplace=True)

In [ ]:
clear_dataset.rename(columns={'http':'image_path'},inplace=True)

**Note :** I am taking those movies whose vote_count >1000
*   you can take own choice according to processor




In [ ]:
clear_dataset=clear_dataset[clear_dataset['vote_count']>1000]

**Content Base recommendation**

In [140]:
content=clear_dataset[['id','title','overview','tagline','genres','keywords']]

In [5]:
content['keywords'][0]

'rescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy, allegory, manipulation, car crash, heist, memory, architecture, los angeles, california, dream world, subconscious'

In [6]:
content['overview'][0]

'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.'

In [42]:
content['tagline'][0]

'Your mind is the scene of the crime.'

In [43]:
content['genres'][0]

'Action, Science Fiction, Adventure'

In [141]:
content['title'].duplicated().sum()

np.int64(74)

In [142]:
content.drop_duplicates(inplace=True)

<ipython-input-142-db2f66d8839c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content.drop_duplicates(inplace=True)


In [143]:
content['content_description']=content['overview']+' '+content['tagline']+' '+content['genres']+' '+content['keywords']

<ipython-input-143-fce8f0d1d9cf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['content_description']=content['overview']+' '+content['tagline']+' '+content['genres']+' '+content['keywords']


In [144]:
content.drop(['overview','tagline','genres','keywords'],axis=1,inplace=True)

<ipython-input-144-cdf6bac8cb6a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content.drop(['overview','tagline','genres','keywords'],axis=1,inplace=True)


In [57]:
content.head()

,id,title,content_description
0,27205,Inception,"Cobb, a skilled thief who commits corporate es..."
1,157336,Interstellar,The adventures of a group of explorers who mak...
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...


In [145]:
lemi=WordNetLemmatizer()

In [146]:
def cleaning(text):
  text=text.lower()
  text=text.translate(str.maketrans('','', string.punctuation))
  text_tokens=word_tokenize(text)
  filtered_text=[word for word in text_tokens if word not in stopwords.words('english')]
  lemmatized_text=[lemi.lemmatize(word) for word in filtered_text]
  return ' '.join(filtered_text)

In [147]:
content['clean_content_description']=content['content_description'].apply(cleaning)

<ipython-input-147-9d3acc0042cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['clean_content_description']=content['content_description'].apply(cleaning)


In [148]:
content.drop(['content_description'],axis=1,inplace=True)

<ipython-input-148-629f6356b9e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content.drop(['content_description'],axis=1,inplace=True)


In [149]:
content.drop(['id'],axis=1,inplace=True)

<ipython-input-149-e726aa52d302>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content.drop(['id'],axis=1,inplace=True)


In [150]:
content['title']=content['title'].str.lower()

<ipython-input-150-3c1d5c8674b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['title']=content['title'].str.lower()


In [152]:
vector = TfidfVectorizer(stop_words='english')

In [153]:
tf_idf_vector=vector.fit_transform(content['clean_content_description'])

In [154]:
cosine_score=cosine_similarity(tf_idf_vector,tf_idf_vector)

In [155]:
cosine_score.shape

(3638, 3638)

In [189]:
clear_dataset=clear_dataset[['title','vote_average','status','release_date','revenue','runtime','adult','budget','original_language','overview','image_path']]

In [182]:
def content_recomend(movie):
  movie_index = content[content['title']==movie].index[0]
  similar_movie = list(enumerate(cosine_score[movie_index]))
  sorted_similar_movies = sorted(similar_movie, key=lambda x: x[1], reverse=True)[1:6]
  temp_data=[]
  for i in sorted_similar_movies:
    temp=clear_dataset.iloc[i[0]]
    temp_store=[
        temp['title'],
        temp['vote_average'],
        temp['status'],
        temp['release_date'],
        temp['revenue'],
        temp['runtime'],
        temp['adult'],
        temp['budget'],
        temp['original_language'],
        temp['overview']
    ]
    temp_data.append(temp_store)
  return temp_data


In [183]:
content_recomend('red dragon')

[['Manhunter',
  np.float64(7.0),
  'Released',
  '1986-08-14',
  np.int64(8620929),
  np.int64(120),
  np.False_,
  np.int64(15000000),
  'en',
  'FBI Agent Will Graham, who retired after catching Hannibal Lecter, returns to duty to engage in a risky cat-and-mouse game with Lecktor to capture a new killer.'],
 ['The Silence of the Lambs',
  np.float64(8.3),
  'Released',
  '1991-02-14',
  np.int64(272742922),
  np.int64(119),
  np.False_,
  np.int64(19000000),
  'en',
  "Clarice Starling is a top student at the FBI's training academy.  Jack Crawford wants Clarice to interview Dr. Hannibal Lecter, a brilliant psychiatrist who is also a violent psychopath, serving life behind bars for various acts of murder and cannibalism.  Crawford believes that Lecter may have insight into a case and that Starling, as an attractive young woman, may be just the bait to draw him out."],
 ['Solace',
  np.float64(6.4),
  'Released',
  '2015-09-03',
  np.int64(22586863),
  np.int64(101),
  np.False_,
  np

In [161]:
import pickle
pickle.dump(cosine_score,open('cosine_scores.pkl','wb'))
pickle.dump(clear_dataset,open('movies_data.pkl','wb'))